# IMPORTANT!
Remember to change the runtime to "T4 GPU" (Not optional)
On the left upper pert:
Runtimme -> Change runtime tim

# Part 0: required packages

In [ ]:
#  Install necessary libraries, qith -q to be quiet
!pip install transformers torch torchaudio -q

In [ ]:
#  Install necessary libraries, with -q to be quiet
!pip pydub huggingface_hub librosa soundfile yt-dlp sentencepiece  -q

ERROR: unknown command "pydub"


In [ ]:
# some libraries are needed in order to install pyaudio.
!apt install  -qq libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio pydub  evaluate jiwer -q

libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
libportaudio2 is already the newest version (19.6.0-1.1).
libportaudiocpp0 is already the newest version (19.6.0-1.1).
portaudio19-dev is already the newest version (19.6.0-1.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install ffmpeg-python -q

In [ ]:
#Import libraries
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
import pyaudio
import torch
import wave
import librosa
import soundfile as sf
from pydub import AudioSegment
from IPython.display import Audio
import os
from scipy.signal import resample
from datasets import load_dataset

In [ ]:
# This part seems necessary for later

import evaluate
from jiwer import cer

def down_sample_audio(audio_original, original_sample_rate):
    target_sample_rate = 16000

    # Calculate the number of samples for the target sample rate
    num_samples = int(len(audio_original) * target_sample_rate / original_sample_rate)

    # Resample the audio array to the target sample rate
    downsampled_audio = resample(audio_original, num_samples)

    return downsampled_audio

# Part 1: Gettiong models ready

## Load model

In [ ]:
# Load every part of the small model separately

MODEL_REPO = "SchindleriaPraematurus/whisper-tiny-gn-finetuned" # Your repository
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


SMALL_TOKENIZER_COMMIT_HASH = "6cd67df1b6923465434cee5bb80aadfe7a8d74ab"  # Commit for "Upload tokenizer"
SMALL_FEATURE_EXTRACTOR_COMMIT_HASH = "a7f430d0d46753348597fb241734d341c48fc69b" # Commit for "Upload feature_extractor"
SMALL_MODEL_COMMIT_HASH = "06294d5a001f1f32290e98cf9a4a6006f1fc394a" # Commit for "Upload fine-tuned Whisper-small"

In [ ]:


print(f"Attempting to load 'small' version components from specific commits:")
print(f" - Tokenizer commit: {SMALL_TOKENIZER_COMMIT_HASH}")
print(f" - Feature Extractor commit: {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}")
print(f" - Model commit: {SMALL_MODEL_COMMIT_HASH}")

tokenizer_small = None
feature_extractor_small = None
model_small = None
processor_small = None

try:
    # Load Tokenizer
    # These settings are usually saved in tokenizer_config.json.
    print(f"\nLoading small tokenizer from revision: {SMALL_TOKENIZER_COMMIT_HASH}...")
    tokenizer_small = WhisperTokenizer.from_pretrained(
        MODEL_REPO,
        revision=SMALL_TOKENIZER_COMMIT_HASH
    )
    print("Successfully loaded 'small' tokenizer.")

    # Load Feature Extractor
    print(f"\nLoading small feature extractor from revision: {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}...")
    feature_extractor_small = WhisperFeatureExtractor.from_pretrained(
        MODEL_REPO,
        revision=SMALL_FEATURE_EXTRACTOR_COMMIT_HASH
    )
    print("Successfully loaded 'small' feature extractor.")

    # Load Model
    print(f"\nLoading small model from revision: {SMALL_MODEL_COMMIT_HASH}...")
    model_small = WhisperForConditionalGeneration.from_pretrained(
        MODEL_REPO,
        revision=SMALL_MODEL_COMMIT_HASH
    ).to(DEVICE)
    model_small.eval()
    print("Successfully loaded 'small' model.")

    # Manually create the WhisperProcessor instance
    if tokenizer_small and feature_extractor_small:
        processor_small = WhisperProcessor(feature_extractor=feature_extractor_small, tokenizer=tokenizer_small)
        print("\nSuccessfully created 'small' WhisperProcessor manually.")
    else:
        print("\nCould not create 'small' processor because tokenizer or feature extractor failed to load.")

except Exception as e:
    print(f"\nAn error occurred during loading of 'small' components: {e}")
    print("Please double-check the following:")
    print("1. All three commit hashes are correct and point to the respective uploads for the 'small' version.")
    print(f"2. The tokenizer files (vocab.json, tokenizer_config.json, etc.) are indeed present at commit {SMALL_TOKENIZER_COMMIT_HASH}.")
    print(f"3. The feature extractor's preprocessor_config.json is present at commit {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}.")
    print(f"4. The model files (pytorch_model.bin, config.json) are present at commit {SMALL_MODEL_COMMIT_HASH}.")

if model_small and processor_small:
    print(f"\nSuccessfully prepared 'small' model (from multiple commits) for comparison.")
    # You can now use model_small and processor_small for predictions
else:
    print(f"\nFailed to fully load the 'small' model and processor. Check error messages above.")

Attempting to load 'small' version components from specific commits:
 - Tokenizer commit: 6cd67df1b6923465434cee5bb80aadfe7a8d74ab
 - Feature Extractor commit: a7f430d0d46753348597fb241734d341c48fc69b
 - Model commit: 06294d5a001f1f32290e98cf9a4a6006f1fc394a

Loading small tokenizer from revision: 6cd67df1b6923465434cee5bb80aadfe7a8d74ab...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Successfully loaded 'small' tokenizer.

Loading small feature extractor from revision: a7f430d0d46753348597fb241734d341c48fc69b...
Successfully loaded 'small' feature extractor.

Loading small model from revision: 06294d5a001f1f32290e98cf9a4a6006f1fc394a...
Successfully loaded 'small' model.

Successfully created 'small' WhisperProcessor manually.

Successfully prepared 'small' model (from multiple commits) for comparison.


In [ ]:
# Controls how the model produces the output sequence of tokens.
model_small.generation_config.language = "spanish"
model_small.generation_config.task = "transcribe"
model_small.generation_config.forced_decoder_ids = None

gen_config_for_small = model_small.generation_config


## Handle translation

In [ ]:
# TRANSLATING PART

from transformers import pipeline

# --- Models for Guarani to Spanish Translation ---

# Larger, potentially more accurate model: NLLB (No Language Left Behind)
# NLLB models are powerful but can be large. We'll try a distilled (smaller) version.
translator_gn_es_nllb = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="grn_Latn", # Guarani using Latin script
    tgt_lang="spa_Latn",  # Spanish using Latin script
    device='cuda'
)

# Guarani to englsih
translator_gn_en_nllb = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="grn_Latn", # Guarani using Latin script
    tgt_lang="eng_Latn",  # Spanish using Latin script
    device='cuda'
)



Device set to use cuda
Device set to use cuda


## Feed the audio in little chunks
This way, it has better results

You can modify the size of the chunks in the variable "chunk_duration_sec"

In [ ]:
def transcribe_long_audio_segmented(
    full_audio_path,
    model,
    feature_extractor,
    tokenizer,
    gen_config,
    chunk_duration_sec=5,  # Whisper generally works well with segments up to 30s.
    overlap_sec=2,          # Overlap helps avoid cutting words/sentences and provides context.
    target_sr=16000
):
    """
    Loads a long audio file, chunks it, and transcribes each chunk sequentially.
    """
    all_transcriptions = []

    try:
        print(f"Loading and resampling audio file: {full_audio_path} to {target_sr}Hz...")
        audio_full, sr_orig = librosa.load(full_audio_path, sr=None, mono=True)
        if sr_orig != target_sr:
            audio_full = librosa.resample(y=audio_full, orig_sr=sr_orig, target_sr=target_sr)
        print(f"Audio loaded. Duration: {len(audio_full)/target_sr:.2f}s, Sample rate: {target_sr}Hz")

        chunk_len_samples = int(chunk_duration_sec * target_sr)
        overlap_samples = int(overlap_sec * target_sr)
        step_len_samples = chunk_len_samples - overlap_samples

        # If the audio is shorter than or equal to one chunk duration, process it as a single segment
        if len(audio_full) <= chunk_len_samples:
            print("Audio is shorter than or equal to one configured chunk, processing as a single segment...")
            if len(audio_full) < int(target_sr * 0.1): # Skip if audio is extremely short (e.g. <0.1s)
                 print(f"Audio too short ({len(audio_full)/target_sr:.2f}s), skipping.")
                 return "[Audio too short to process]"
            single_transcription = process_audio_chunk(audio_full, model, feature_extractor, tokenizer, gen_config)
            return single_transcription

        # Validate chunking parameters for longer audio
        if step_len_samples <= 0:
            print(f"Error: Chunk duration ({chunk_duration_sec}s) must be greater than overlap ({overlap_sec}s).")
            print("Cannot proceed with current chunking parameters.")
            return "[Error: Invalid chunking parameters (chunk_duration_sec > overlap_sec required)]"

        start_sample = 0
        num_processed_chunks = 0
        while start_sample < len(audio_full):
            num_processed_chunks += 1
            end_sample = start_sample + chunk_len_samples

            # Define the current chunk
            audio_chunk = audio_full[start_sample:min(end_sample, len(audio_full))]

            # If the current chunk (especially a leftover segment) is too short, skip it
            if len(audio_chunk) < int(target_sr * 0.5): # Minimum 0.5 seconds for a chunk
                if num_processed_chunks == 1: # If the whole audio was deemed too short initially by this check
                    print(f"Entire audio effectively too short after initial checks ({len(audio_chunk)/target_sr:.2f}s).")

                else:
                    print(f"Skipping very short remaining segment ({len(audio_chunk)/target_sr:.2f}s) at the end.")
                break

            #print(f"\nProcessing chunk {num_processed_chunks}: "
                  #f"Time {start_sample/target_sr:.2f}s to {min(end_sample, len(audio_full))/target_sr:.2f}s")

            chunk_transcription = process_audio_chunk(
                audio_chunk, model, feature_extractor, tokenizer, gen_config
            )

            if chunk_transcription:
                #print(f"Chunk {num_processed_chunks} transcription: {chunk_transcription}")
                all_transcriptions.append(chunk_transcription)
            else:
                print(f"Chunk {num_processed_chunks} yielded no transcription.")

            # Move to the start of the next chunk
            start_sample += step_len_samples

        # Join the transcriptions from all chunks
        # A simple space join is used here. For more advanced stitching,
        final_transcription = " ".join(all_transcriptions)
        return final_transcription

    except Exception as e:
        print(f"An error occurred during segmented transcription of {full_audio_path}: {e}")
        # import traceback # Uncomment for detailed traceback
        # traceback.print_exc() # Uncomment for detailed traceback
        return None

In [ ]:
import torch
import librosa
import numpy as np


def process_audio_chunk(audio_chunk_16khz, model, feature_extractor, tokenizer, gen_config):
    """
    Transcribes a single audio chunk (NumPy array at 16kHz mono) using the provided Whisper components.
    """
    try:
        # 1. Extract input features
        input_features_pt = feature_extractor(
            raw_speech=audio_chunk_16khz,
            sampling_rate=16000,
            return_tensors='pt'
        ).input_features

        # 2. Move features to the model's device
        device = model.device
        input_features_pt = input_features_pt.to(device)

        # 3. Generate transcription
        with torch.no_grad():
            predicted_ids = model.generate(input_features_pt, generation_config=gen_config)
            # If you want to be more explicit with language and task for Whisper:
            # predicted_ids = model.generate(
            #     input_features_pt,
            #     language=gen_config.language if hasattr(gen_config, 'language') else "your_language_code", # e.g., "es" for Spanish
            #     task=gen_config.task if hasattr(gen_config, 'task') else "transcribe"
            # )


        # 4. Decode the predicted tokens
        transcription = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        return transcription.strip()

    except Exception as e:
        print(f"An error occurred during transcription of an audio chunk: {e}")
        # import traceback
        # traceback.print_exc()
        return "" # Return empty string for failed chunks

# Part 3: getting audio

In [ ]:
# Cell 4: Record audio for a fixed duration
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1              # Number of audio channels (1 for mono, 2 for stereo)
RATE = 16000              # Sample rate (Whisper expects 16kHz)
CHUNK = 1024              # Number of frames per buffer
RECORD_SECONDS = 10        # Duration of recording in seconds
WAV_FILENAME = "recorded_audio.wav"
MP3_FILENAME = "recorded_audio.mp3"

audio = pyaudio.PyAudio()

In [ ]:
import yt_dlp
import subprocess

def download_youtube_audio_segment(video_url, start_time_str, end_time_str, output_filename="audio.mp3"):
    """
    Downloads a specific segment of audio from a YouTube video as an MP3 file.

    Args:
        video_url (str): The URL of the YouTube video.
        start_time_str (str): The start time of the segment (e.g., "00:01:30" for 1 minute 30 seconds).
                              If None, downloads from the beginning.
        end_time_str (str): The end time of the segment (e.g., "00:02:00" for 2 minutes).
                            If None, downloads until the end.
        output_filename (str, optional): The desired name for the output MP3 file.
                                         Defaults to "audio.mp3".
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',  # Temporary filename
        'quiet': True,
    }

    if start_time_str and end_time_str:
        # yt-dlp uses ffmpeg for time range extraction during postprocessing
        # We can also specify download sections directly if the server supports it (less common for precise cuts)
        # A more robust way is to download the best audio and then cut it with ffmpeg.

        # Download the full audio first (or best available section if supported by extractor)
        ydl_opts_full_audio = {
            'format': 'bestaudio/best',
            'outtmpl': f'%(id)s_full.%(ext)s',
            'quiet': True,
        }
        temp_filename = ""
        video_id = ""
        try:
            with yt_dlp.YoutubeDL(ydl_opts_full_audio) as ydl:
                info = ydl.extract_info(video_url, download=True)
                video_id = info.get('id', 'temp_video')
                # Construct the downloaded filename based on ydl_opts_full_audio
                # This assumes the extension is one of the common audio ones yt-dlp might download
                # Common extensions: m4a, webm, ogg, mp3 (though conversion to mp3 is a post-processing step)
                # We'll try to find the actual downloaded extension.
                downloaded_ext = info.get('ext', 'm4a') # Default to m4a if not found
                temp_filename = f"{video_id}_full.{downloaded_ext}"


            # Use ffmpeg to cut the segment and convert to mp3
            ffmpeg_command = [
                'ffmpeg',
                '-hide_banner', # Suppress version and build info
                '-loglevel', 'error', # Suppress all logs except errors
                '-y', #overwrite file if it exists
                '-i', temp_filename,
                '-ss', start_time_str,
                '-to', end_time_str,
                '-vn',  # No video
                '-acodec', 'libmp3lame', # Audio codec MP3
                '-q:a', '2',  # Audio quality (0-9, lower is better for VBR)
                output_filename
            ]
            subprocess.run(ffmpeg_command, check=True)
            print(f"Audio segment saved as {output_filename}")
        except yt_dlp.utils.DownloadError as e:
            print(f"Error during download: {e}")
        except subprocess.CalledProcessError as e:
            print(f"Error during ffmpeg processing: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        finally:
            # Clean up the temporary full audio file
            if temp_filename and os.path.exists(temp_filename):
                os.remove(temp_filename)

    else: # Download the whole audio as mp3
        ydl_opts.update({
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192', # Standard MP3 quality
            }],
            'outtmpl': output_filename.replace('.mp3', ''), # yt-dlp adds the .mp3
        })
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])
            print(f"Full audio saved as {output_filename}")
        except Exception as e:
            print(f"An error occurred: {e}")


## Record audio
You c press the button when you finished speaking. Your recording will be stored as mp3 for later use

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/questions/18650168/convert-blob-to-base64-without-formdata
https://stackoverflow.com/questions/12710001/how-to-convert-uint8-array-to-base64-encoded-string/12713326#12713326
Or maybe superficially somewhere else I can't find. The important thing is that it works.
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 16000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data promise, returning:" + base64data);
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio(WAV_FILENAME="recorded_audio.wav"):
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output(WAV_FILENAME, format='wav', ac=1, ar='16000') # Ensure 16kHz mono
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  # If you want to play it back in Colab to verify
  # display(Audio(WAV_FILENAME))
  return WAV_FILENAME

# How to use it:
WAV_FILENAME = "my_recorded_colab_audio.wav"
get_audio(WAV_FILENAME) # This will trigger the recording UI
print(f"Audio recorded and saved to {WAV_FILENAME}")

try:
    sound = AudioSegment.from_wav(WAV_FILENAME)
    sound.export(MP3_FILENAME, format="mp3")
    print(f"Converted to {MP3_FILENAME}")
    os.remove(WAV_FILENAME) # Optional: remove the intermediate WAV file
    # Display the audio player in the notebook
    display(Audio(MP3_FILENAME))
# Now you can load WAV_FILENAME with librosa or soundfile for Whisper
# import librosa
# audio_data, sr = librosa.load(WAV_FILENAME, sr=16000) # Whisper expects 16kHz
# print(f"Loaded {WAV_FILENAME}, Sample rate: {sr}, Duration: {len(audio_data)/sr:.2f}s")
except Exception as e:
    print(f"Could not convert to MP3. Make sure ffmpeg is installed and in your PATH. Error: {e}")
    print("You can still use the WAV file for prediction if MP3 conversion fails, but ensure your prediction block loads the correct file type.")

Audio recorded and saved to my_recorded_colab_audio.wav
Converted to recorded_audio.mp3


## Get audio from YouTube
You can also choose YouTube as the source of your input

In [ ]:
#Get audio from YouTube
import os # For cleanup in the example

video_url = "https://www.youtube.com/watch?v=17EoMoyS1YM&ab_channel=MediosdelEstado" # Replace with a valid YouTube URL

# Example 1: Download a specific segment
start_segment = "00:00:09"
end_segment = "00:00:30"
output_segment_filename = "youtube_segment.mp3"
print(f"\nDownloading segment from {start_segment} to {end_segment}...")
download_youtube_audio_segment(video_url, start_segment, end_segment, output_segment_filename)

# Example 2: Download the whole audio (default behavior if start/end are None)
# output_full_filename = "full_audio.mp3"
# print(f"\nDownloading full audio...")
# download_youtube_audio_segment(video_url, None, None, output_full_filename)

# Example 3: Specify only start time (will download till the end from start_time)
# Note: The current function structure requires both start and end for segmentation,
# or neither for full download. To support only start_time and download till end,
# you'd need to get video duration first and use that as end_time_str.
# For simplicity, this example is omitted but can be added by fetching video duration using yt_dlp.

# Example 4: Specify output filename for full download
# custom_full_output = "my_favorite_song_full.mp3"
# print(f"\nDownloading full audio to a custom filename...")
# download_youtube_audio_segment(video_url, None, None, custom_full_output)


Audio segment saved as youtube_segment.mp3


# Part 4: Use the model!

### Run this to transcribe YouTube audio!
Comment the youtube line, and uncomment the recorded_audio.mp3 line to switch to YouOwn audio!



In [ ]:
my_model=model_small
my_feature_extractor=feature_extractor_small
my_tokenizer=tokenizer_small
my_gen_config=gen_config_for_small

In [ ]:
input_audio_file = "youtube_segment.mp3" # Your target MP3 file
#input_audio_file = "your_recording.mp3" # Your target MP3 file

if 'my_model' in locals() and 'my_feature_extractor' in locals() and \
   'my_tokenizer' in locals() and 'my_gen_config' in locals():

    print(f"\nAttempting to transcribe long audio file (segmented): {input_audio_file}")

    # You can adjust chunk_duration_sec and overlap_sec here if needed
    predicted_text = transcribe_long_audio_segmented(
        input_audio_file,
        my_model,
        my_feature_extractor,
        my_tokenizer,
        my_gen_config,
        chunk_duration_sec=4, # e.g., 28 seconds per chunk
        overlap_sec=0.25           # e.g., 2 seconds overlap
    )

    if predicted_text is not None:
        print("\n--- Final Predicted Transcription (Segmented) ---")
        print(predicted_text)
    else:
        print("\nSegmented transcription failed.")

else:
    print("Please ensure 'my_model', 'my_feature_extractor', 'my_tokenizer', and 'my_gen_config' are defined.")


Attempting to transcribe long audio file (segmented): youtube_segment.mp3
Loading and resampling audio file: youtube_segment.mp3 to 16000Hz...


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.


Audio loaded. Duration: 21.00s, Sample rate: 16000Hz


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensA


--- Final Predicted Transcription (Segmented) ---
Ore Ru yvágape reivéva toñembojoporã. Tujeruviákena nde réra. tojejapo ne rembipota ko yvy ári yvahapeguáicha Eme'ẽ oréve ko árape rembi'urã opa araroakotevẽvéva Ehejareíkena oréve rembiapo vaikue rohejareiháichorõ rapicha. Tapichápe hembiapo vaipo rendive.


In [ ]:
asr_text_guarani = predicted_text

translation_gn_en_nllb = translator_gn_en_nllb(asr_text_guarani)
print(f"English: {translation_gn_en_nllb[0]['translation_text']}")

translation_gn_es_nllb = translator_gn_es_nllb(asr_text_guarani)
print(f"Spanish: {translation_gn_es_nllb[0]['translation_text']}")

English: "Our Father in heaven, hallowed be thy name. Thy will be done on earth as on the earth. Give us this day our daily bread. Forgive us our trespasses, for we have forgiven our sins.
Spanish: Padre nuestro que estás en los cielos, que tu voluntad sea santificada, que tu nombre sea santificado, que tu voluntad sea hecha en la tierra como en los jardines, dannos hoy nuestro alimento de cada día.


##